In [ ]:
import pandas as pd
import torch
import numpy as np
import cv2
import matplotlib.pyplot as plt
import keras

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json
api_token = {"username":"yishengliu","key":"600d4a6327f17945c27ff54c8bae7bad"}
import json
with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c challenges-in-representation-learning-facial-expression-recognition-challenge
!unzip /content/challenges-in-representation-learning-facial-expression-recognition-challenge.zip
!rm -rf /content/train.csv /content/test.csv /content/icml_face_data.csv /content/example_submission.csv /content/challenges-in-representation-learning-facial-expression-recognition-challenge.zip
!tar -xzf /content/fer2013.tar.gz
!mkdir dataset
!mkdir dataset/fer2013
!mv /content/fer2013/fer2013.csv /content/dataset/fer2013
!rm -rf /content/fer2013 /content/fer2013.tar.gz

In [ ]:
class dataset_provider(object):
    def __init__(self, img_size):
        self.img_size = img_size

    def dataset_initializer(self):
        pixels_from_csv = pd.read_csv('/content/dataset/fer2013/fer2013.csv')
        pixels_list = pixels_from_csv['pixels'].tolist()
        ds_emotions = pd.get_dummies(pixels_from_csv['emotion']).values
        ds_faces = []
        for pixel_in_sequence in pixels_list:
            face = [int(pixel) for pixel in pixel_in_sequence.split(' ')]
            face = np.asarray(face).reshape(48, 48)
            face = cv2.resize(face.astype('uint8'), self.img_size)
            ds_faces.append(face.astype('float32'))
        ds_faces = np.expand_dims(np.asarray(ds_faces), -1)
        return ds_faces, ds_emotions

In [ ]:
dataset = dataset_provider((64,64))
faces, emotions = dataset.dataset_initializer()
faces = faces.astype('float32')
faces = ((faces / 255.0) - 0.5) * 2.0
n_train = int(0.8 * len(faces))
n_test = len(faces) - n_train
ds_train_faces = faces[:n_train]
ds_train_emotions = emotions[:n_train]
ds_test_faces = faces[n_train:]
ds_test_emotions = emotions[n_train:]

In [ ]:
def base_model(input_shape, classes_no):
  input_data = keras.layers.Input(input_shape)
  model = keras.layers.Convolution2D(8,(3,3))(input_data)
  model = keras.layers.BatchNormalization()(model)
  model = keras.layers.Activation('relu')(model)
  my_filters = [16,64,256,512]
  for filter in my_filters:
    model = keras.layers.Convolution2D(filter,(3,3),padding='same')(model)
    model = keras.layers.BatchNormalization()(model)
    model = keras.layers.Activation('relu')(model)
    model = keras.layers.Convolution2D(filter,(3,3),padding='same')(model)
    model = keras.layers.BatchNormalization()(model)
    model = keras.layers.MaxPooling2D(pool_size=(2,2), padding='same')(model)
    model = keras.layers.Activation('relu')(model)
  model = keras.layers.Convolution2D(classes_no,(3,3),padding='same')(model)
  model = keras.layers.GlobalMaxPooling2D()(model)
  ret_model = keras.models.Model(input_data, keras.layers.Activation('softmax',name='predictions')(model))
  return ret_model

In [ ]:
input_shape = (64,64,1)
classes_no = 7
batch = 4
model = base_model(input_shape, classes_no)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(epochs=100,
     x=keras.preprocessing.image.ImageDataGenerator(
     horizontal_flip=True,
     rotation_range=15,
     width_shift_range=0.1,
     height_shift_range=0.1,
     zoom_range=0.1
     ).flow(ds_train_faces,ds_train_emotions,batch),
     validation_data=(ds_test_faces,ds_test_emotions),
     steps_per_epoch=len(ds_train_faces)/batch)